In [1]:
import numpy as np

from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.topology.molecule import Molecule
from openforcefield.topology.topology import Topology

from system.typing.smirnoff import *
from system.system import System

In [2]:
# Load in a mainline OpenFF forcefield and construct a minimal ethanol topology
parsley = ForceField('openff-1.0.0.offxml')

top =  Topology.from_molecules(Molecule.from_smiles('CCO'))

In [3]:
# Generate a (partial) mapping of topology slots and SMIRNOFF SMIRKS
slot_smirks_map = build_slot_smirks_map(forcefield=parsley, topology=top)

slot_smirks_map

{'Constraints': {(0, 3): '[#1:1]-[*:2]',
  (0, 4): '[#1:1]-[*:2]',
  (0, 5): '[#1:1]-[*:2]',
  (1, 6): '[#1:1]-[*:2]',
  (1, 7): '[#1:1]-[*:2]',
  (2, 8): '[#1:1]-[*:2]'},
 'Bonds': {(0, 1): '[#6X4:1]-[#6X4:2]',
  (0, 3): '[#6X4:1]-[#1:2]',
  (0, 4): '[#6X4:1]-[#1:2]',
  (0, 5): '[#6X4:1]-[#1:2]',
  (1, 2): '[#6:1]-[#8:2]',
  (1, 6): '[#6X4:1]-[#1:2]',
  (1, 7): '[#6X4:1]-[#1:2]',
  (2, 8): '[#8:1]-[#1:2]'},
 'Angles': {(0, 1, 2): '[*:1]~[#6X4:2]-[*:3]',
  (0, 1, 6): '[*:1]~[#6X4:2]-[*:3]',
  (0, 1, 7): '[*:1]~[#6X4:2]-[*:3]',
  (1, 0, 3): '[*:1]~[#6X4:2]-[*:3]',
  (1, 0, 4): '[*:1]~[#6X4:2]-[*:3]',
  (1, 0, 5): '[*:1]~[#6X4:2]-[*:3]',
  (1, 2, 8): '[*:1]-[#8:2]-[*:3]',
  (2, 1, 6): '[*:1]~[#6X4:2]-[*:3]',
  (2, 1, 7): '[*:1]~[#6X4:2]-[*:3]',
  (3, 0, 4): '[#1:1]-[#6X4:2]-[#1:3]',
  (3, 0, 5): '[#1:1]-[#6X4:2]-[#1:3]',
  (4, 0, 5): '[#1:1]-[#6X4:2]-[#1:3]',
  (6, 1, 7): '[#1:1]-[#6X4:2]-[#1:3]'},
 'ProperTorsions': {(0, 1, 2, 8): '[#6X4:1]-[#6X4:2]-[#8X2H1:3]-[#1:4]',
  (2, 1, 0, 3): '

In [4]:
# This is not efficient at the moment, will take a minute or so to run
smirks_potential_map = build_smirks_potential_map(forcefield=parsley, smirks_map=smirks_map)

NameError: name 'smirks_map' is not defined

In [ ]:
# This is like the smirnoff_collection in the other notebook, but encompasses more parameters
# (all vdW and bond parameters in the force field). You can inspect, for example, the list
# of all SMIRKS patterns of bonds
smirks_potential_map['Bonds'].keys()

In [ ]:
# Or look up a specific parameter within a particlar bond
smirks_potential_map['Bonds']['[#6X4:1]-[#1:2]'].parameters['k']

In [ ]:
# TODO: Deal with case of system only partially being built up
# Can build up a system object using the data we have
ethanol_system = System(
    forcefield=parsley,
    topology=top,
    positions=np.zeros(shape=(top.n_topology_atoms, 3)),
    box=np.zeros((3, 3)),
)

In [ ]:
# And using the SMIRKS map (which should be merged into the System at some point), we can
# find the SMIRKS that was assigned to an arbitrary bond, say index 4
bond_key = tuple([a.topology_atom_index for a in ethanol_system.topology.bond(4).atoms])
smirks_map['Bonds'][bond_key]

In [ ]:
# And then using that SMIRKS as a "key", we can look up its associated potential
ethanol_system.potential_collection['Bonds'][smirks_map['Bonds'][bond_key]]

In [ ]:
ethanol_system.potential_collection